# Initial data read
Notebook to begin exploring the UK air pollution data made available by the department for environment, food and rural affairs: [UK Air Information Resource](https://uk-air.defra.gov.uk/).  
&copy; Crown 2020 copyright Defra via uk-air.defra.gov.uk, licenced under the [Open Government Licence (OGL)](http://www.nationalarchives.gov.uk/doc/open-government-licence/version/2/).  
  
First, we will inspect air pollution readings from a measurement station in Oxford, UK.  
  
Notebook successfully:
- Achieves initial read-in of air pollution data from Oxford measurement station
- Point 2

In [1]:
import sys
import time
from pathlib import Path
import pandas as pd
import numpy as np
import altair as alt

# auto-formatting of notebook cell code
%load_ext lab_black

repo_location = (
    "\\Users\\Simon\\Documents\\Python_Projects\\Productivity Boiz\\air-pollution"
)

sys.path.append(repo_location)
top_level_dir = Path(repo_location)
interim_data_path = top_level_dir / "data/interim"

# if not interim_data_path.is_dir():
#     interim_data_path.mkdir()
#
# save_location = interim_data_path / "oxford_ebbes_full.csv"
# df.to_csv(save_location, index=False)
# print(f"Full historic air pollution data for {} ({time_range}) saved to: {save_location}")

# tracks version numbers used to run notebook
%load_ext watermark
%watermark -u -n -t -iv -v -g -a Simon-Lee-UK

pandas 1.0.3
numpy  1.18.2
altair 4.0.1
Simon-Lee-UK 
last updated: Sat Mar 28 2020 16:14:03 

CPython 3.7.3
IPython 7.13.0
Git hash: 6be5bccd32729fc52e6bb3cfddc34da1306c5142


## Data info
Location: Oxford St Ebbes (OX8)
All Data GMT hour ending  
Status: R = Ratified / P = Provisional / P* = As supplied  
Data available: 2008 - 2020
  
### Raw columns (2020):
`Date` - format: 'dd-mm-yyyy'  
`time` - 24 hour, e.g. '19:00'  
`PM<sub>10</sub> particulate matter (Hourly measured)` - $PM_{10}$ particulate matter with diameter $< 10 \;\mu m$  
`status` - ratified / provisional / as supplied  
`unit` - $\mu g \:/\: m^{3}$ (FIDAS)  
`Nitric oxide` - 'NO'  
`status.1` - ratified / provisional / as supplied  
`unit.1` - $\mu g \:/\: m^{3}$  
`Nitrogen dioxide` - 'NO$_2$'  
`status.2` - ratified / provisional / as supplied  
`unit.2` - $\mu g \:/\: m^{3}$  
`Nitrogen oxides as nitrogen dioxide` - 'NO$_x$ as NO$_2$'  
`status.3` - ratified / provisional / as supplied  
`unit.3` - $\mu g \:/\: m^{3}$  
`PM<sub>2.5</sub> particulate matter (Hourly measured)` - $PM_{2.5}$ particulate matter with diameter $< 2.5 \;\mu m$    
`status.4` - ratified / provisional / as supplied  
`unit.4` - $\mu g \:/\: m^{3}$ (Ref.eq)  
  
#### Additional columns found in data from previous years
`Volatile PM<sub>2.5</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 2.5 \;\mu m$)  
`Volatile PM2.5 (Hourly measured)` - alternate column title for the above; never both observed for same year's data; 'column_consistency' function renames this column to match the above  
`Volatile PM<sub>10</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 10 \;\mu m$)  
`Non-volatile PM<sub>2.5</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 2.5 \;\mu m$)  
`Non-volatile PM<sub>10</sub> (Hourly measured)` - sub-category of particulate matter measurement ($< 10 \;\mu m$)  
  
Volatile/Non-volatile measurements are only available in some years. The two values add to give the generic particulate matter measurement for each hour; this value is available for both sizes in all available years of data. For now, volatile/non-volatile values will be dropped because they cannot be compared across the full range of data.

## Data read functions
Below, define functions for reading and pre-processing the raw data. These functions will later be exported out to a separate package for ease of maintenance and for access from future notebooks.

In [2]:
def get_single_year(year):
    data_url = f"https://uk-air.defra.gov.uk/data_files/site_data/OX8_{year}.csv"
    single_year = pd.read_csv(data_url, header=4)
    return single_year


def column_consistency(raw_data):
    column_consist_dict = {
        "Volatile PM2.5 (Hourly measured)": "Volatile PM<sub>2.5</sub> (Hourly measured)"
    }

    consistent_columns = raw_data.rename(columns=column_consist_dict)
    return consistent_columns


def column_conversion(raw_data):
    column_dict = {
        "Date": "date",
        "PM<sub>10</sub> particulate matter (Hourly measured)": "pm_10",
        "status": "status_pm_10",
        "unit": "unit_pm_10",
        "Nitric oxide": "nitric_oxide",
        "status.1": "status_nitric_oxide",
        "unit.1": "unit_nitric_oxide",
        "Nitrogen dioxide": "nitrogen_dioxide",
        "status.2": "status_nitrogen_dioxide",
        "unit.2": "unit_nitrogen_dioxide",
        "Nitrogen oxides as nitrogen dioxide": "NO2_eq",
        "status.3": "status_NO2_eq",
        "unit.3": "unit_NO2_eq",
        "PM<sub>2.5</sub> particulate matter (Hourly measured)": "pm_2_5",
        "status.4": "status_pm_2_5",
        "unit.4": "unit_pm_2_5",
    }

    converted_columns = raw_data.rename(columns=column_dict)
    converted_columns = extend_date_with_time(converted_columns)
    return converted_columns


def extend_date_with_time(raw_data):
    extended_date = raw_data.copy()
    extended_date.time = extended_date.time.replace("24:00", "00:00")
    extended_date.date = extended_date.date + " " + extended_date.time
    return extended_date


def datetime_conversion(raw_data, target_column="date", date_format="%d-%m-%Y %H:%M"):
    converted_datetime = raw_data.copy()
    try:
        converted_datetime[target_column] = pd.to_datetime(
            converted_datetime[target_column], format=date_format
        )
    except ValueError:
        converted_datetime[target_column] = pd.to_datetime(
            converted_datetime[target_column]
        )

    return converted_datetime

## Combining multiple years of data
Data is available for years 2008 - 2020 with access URLs such as: 'https://uk-air.defra.gov.uk/data_files/site_data/OX8_2008.csv' providing the data for each individual year.  

Use this consistent csv file name format to loop through the available years and save a composite csv file containing all available data.  
Start by defining the list of years we want to download data for:

In [3]:
start_year = 2008
end_year = 2020
years_of_interest = list(np.arange(start_year, end_year + 1))

### Inconsistent column titles between data sets
Following initial read in of some test years of data, some column titles were seen for certain years of data but not others. The full set of observed columns (ignoring associated 'status' and 'unit' columns) was copied to the DataFrame defined in the cell below. This DataFrame is used later to record the presence/absence of each column for each available year of air pollution data. 

In [4]:
inspect_columns = pd.DataFrame(
    {
        "Data (Year)": ["blank"],
        "Date": [False],
        "time": [False],
        "Nitric oxide": [False],
        "Nitrogen dioxide": [False],
        "Nitrogen oxides as nitrogen dioxide": [False],
        "PM<sub>10</sub> particulate matter (Hourly measured)": [False],
        "PM<sub>2.5</sub> particulate matter (Hourly measured)": [False],
        "Volatile PM<sub>10</sub> (Hourly measured)": [False],
        "Volatile PM<sub>2.5</sub> (Hourly measured)": [False],
        "Non-volatile PM<sub>10</sub> (Hourly measured)": [False],
        "Non-volatile PM<sub>2.5</sub> (Hourly measured)": [False],
    }
)
inspect_columns = inspect_columns.append(
    [inspect_columns] * (len(years_of_interest) - 1), ignore_index=True
)

### Looping to read data for years of interest
We also fill the DataFrame defined in the cell above based on the presence/absence of each possible column for each individual year of data after it is read from the source URL.

In [5]:
query_columns = inspect_columns.columns.tolist()

for idx, indv_year in enumerate(years_of_interest):
    single_year = get_single_year(year=indv_year)
    single_year = column_consistency(single_year)

    single_year_cols = single_year.columns.tolist()
    inspect_columns.loc[idx, "Data (Year)"] = indv_year
    for col in query_columns:
        if col in single_year_cols:
            inspect_columns.loc[idx, col] = True

    processed_year = single_year.pipe(column_conversion).pipe(datetime_conversion)
    if idx == 0:
        air_pollution = processed_year.copy()
    else:
        air_pollution = air_pollution.append(processed_year, ignore_index=True)
    time.sleep(1.5)  # creates interval between requests to uk-air.defra.gov.uk

Inspect which columns appear in which years' data sets:

In [6]:
inspect_columns

,Data (Year),Date,time,Nitric oxide,Nitrogen dioxide,Nitrogen oxides as nitrogen dioxide,PM<sub>10</sub> particulate matter (Hourly measured),PM<sub>2.5</sub> particulate matter (Hourly measured),Volatile PM<sub>10</sub> (Hourly measured),Volatile PM<sub>2.5</sub> (Hourly measured),Non-volatile PM<sub>10</sub> (Hourly measured),Non-volatile PM<sub>2.5</sub> (Hourly measured)
0,2008,True,True,True,True,True,True,True,False,True,False,True
1,2009,True,True,True,True,True,True,True,True,True,True,True
2,2010,True,True,True,True,True,True,True,True,True,True,True
3,2011,True,True,True,True,True,True,True,True,True,True,True
4,2012,True,True,True,True,True,True,True,True,True,True,True
5,2013,True,True,True,True,True,True,True,True,True,True,True
6,2014,True,True,True,True,True,True,True,True,True,True,True
7,2015,True,True,True,True,True,True,True,True,True,True,True
8,2016,True,True,True,True,True,True,True,True,True,True,True
9,2017,True,True,True,True,True,True,True,True,True,True,True


Where years have both volatile and non-volatile particulate matter readings: the sum of these two values corresponds to the value in the 'PM<sub>xx</sub>' column (see ['compare particulate matter readings'](#compare-particulate-matter-readings) section below). Happy to just drop these for now (as they're not included in all years). Use the cell below to extract those columns that are to be dropped (data not available for all years)

In [7]:
drop_columns = pd.Series(inspect_columns.all())

### Compare particulate matter readings
Check to ensure that the volatile/non-volatile particulate matter readings sum to give the associated generic value for both 2.5 and 10 $\mu m$ sizes.

In [8]:
volatile = air_pollution.copy()
volatile = volatile[
    [
        "date",
        "Non-volatile PM<sub>2.5</sub> (Hourly measured)",
        "Volatile PM<sub>2.5</sub> (Hourly measured)",
        "pm_2_5",
        "Non-volatile PM<sub>10</sub> (Hourly measured)",
        "Volatile PM<sub>10</sub> (Hourly measured)",
        "pm_10",
    ]
]

In [9]:
volatile.iloc[50000:50010]

,date,Non-volatile PM<sub>2.5</sub> (Hourly measured),Volatile PM<sub>2.5</sub> (Hourly measured),pm_2_5,Non-volatile PM<sub>10</sub> (Hourly measured),Volatile PM<sub>10</sub> (Hourly measured),pm_10
50000,2013-09-14 09:00:00,9.5,0.2,9.7,NaN,NaN,NaN
50001,2013-09-14 10:00:00,9.2,3.0,12.2,11.7,4.1,15.8
50002,2013-09-14 11:00:00,9.2,4.4,13.6,12.5,4.3,16.8
50003,2013-09-14 12:00:00,10.0,3.6,13.6,14.6,4.9,19.5
50004,2013-09-14 13:00:00,13.4,4.3,17.7,17.6,4.4,22.0
50005,2013-09-14 14:00:00,12.5,4.0,16.5,17.7,4.8,22.5
50006,2013-09-14 15:00:00,12.7,5.1,17.8,18.1,6.6,24.7
50007,2013-09-14 16:00:00,11.3,5.1,16.4,19.8,4.2,24.0
50008,2013-09-14 17:00:00,10.6,4.3,14.9,16.4,5.5,21.9
50009,2013-09-14 18:00:00,12.9,4.8,17.7,17.3,6.5,23.8


In [10]:
inspect_columns.columns

Index(['Data (Year)', 'Date', 'time', 'Nitric oxide', 'Nitrogen dioxide',
       'Nitrogen oxides as nitrogen dioxide',
       'PM<sub>10</sub> particulate matter (Hourly measured)',
       'PM<sub>2.5</sub> particulate matter (Hourly measured)',
       'Volatile PM<sub>10</sub> (Hourly measured)',
       'Volatile PM<sub>2.5</sub> (Hourly measured)',
       'Non-volatile PM<sub>10</sub> (Hourly measured)',
       'Non-volatile PM<sub>2.5</sub> (Hourly measured)'],
      dtype='object')

`Volatile PM2.5 (Hourly measured)` column seems to have changed to `Volatile PM<sub>2.5</sub> (Hourly measured)` between 2010 and 2011

Inspect how many missing entries we have for this complete data set:

In [11]:
air_pollution.isna().sum()

date                                                   0
time                                                   0
pm_10                                              21399
status_pm_10                                       21399
unit_pm_10                                         21399
nitric_oxide                                       11981
status_nitric_oxide                                11981
unit_nitric_oxide                                  11981
nitrogen_dioxide                                   13092
status_nitrogen_dioxide                            13092
unit_nitrogen_dioxide                              13092
NO2_eq                                             13092
status_NO2_eq                                      13092
unit_NO2_eq                                        13092
Non-volatile PM<sub>2.5</sub> (Hourly measured)    32558
status_pm_2_5                                      33922
unit_pm_2_5                                        33922
pm_2_5                         

In [12]:
""" Docstring

Parameters
----------
p_1 : dtype
    Description of p_1
p_2 : dtype
    Description of p_2
    
Returns
-------
r_1 : dtype
    Description of r_1
r_2 : dtype
    Description of r_2
"""

' Docstring\n\nParameters\n----------\np_1 : dtype\n    Description of p_1\np_2 : dtype\n    Description of p_2\n    \nReturns\n-------\nr_1 : dtype\n    Description of r_1\nr_2 : dtype\n    Description of r_2\n'